In [70]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2
from net import Net

train_file = 'data/train.mp4'
test_file = 'data/test.mp4'
model_file = 'net.pt'

In [68]:
video_capture = cv2.VideoCapture(test_file)
n_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
imsize = width * height // 16

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time

net = Net(imsize)
net.load_state_dict(torch.load(model_file))

output_notebook()
p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height, title="ehnois")
myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
title = p.title
show(p, notebook_handle=True)
# print(title.text)
end_frame = 5000
for i in range(end_frame): # first 50 frames
    ret, frame = video_capture.read()
    trans_frame = torch.from_numpy(
        cv2.resize(
            cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
            (int(width / 4), int(height / 4))
        ) / 255
    ).float()
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
    frame=cv2.flip(frame, -1)
    myImage.data_source.data['image']=[frame]
    
    pred = net(trans_frame.view(imsize))
    
    title.text = f'Frame {i}, predicted speed: {pred.item()}'
    
    push_notebook()
    time.sleep(0)
    

Loading BokehJS ...